In [1]:
import pandas

df = pandas.read_csv('semantic.tsv', sep='\t', names=['text', 'is_positive'])

df = df.sample(frac=1).reset_index(drop=True)

In [11]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df.text, df.is_positive, test_size=0.2, random_state=10)

In [4]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

/Users/nvkorolkov/.virtualenvs/ml3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=100, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=100, padding='post', truncating='post')

In [6]:
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Deconv2D
from keras import layers
from keras.models import Sequential

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=30,
                    input_length=100))
# model.add(layers.Flatten())
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 1s 550us/step - loss: 0.6936 - acc: 0.4900 - val_loss: 0.6927 - val_acc: 0.5125
Epoch 2/10
1600/1600 [==============================] - 0s 296us/step - loss: 0.6909 - acc: 0.5044 - val_loss: 0.6910 - val_acc: 0.5125
Epoch 3/10
1600/1600 [==============================] - 0s 291us/step - loss: 0.6837 - acc: 0.5112 - val_loss: 0.6884 - val_acc: 0.5300
Epoch 4/10
1600/1600 [==============================] - 0s 283us/step - loss: 0.6697 - acc: 0.6681 - val_loss: 0.6848 - val_acc: 0.5300
Epoch 5/10
1600/1600 [==============================] - 0s 283us/step - loss: 0.6432 - acc: 0.7056 - val_loss: 0.6791 - val_acc: 0.5600
Epoch 6/10
1600/1600 [==============================] - 0s 287us/step - loss: 0.6020 - acc: 0.8319 - val_loss: 0.6726 - val_acc: 0.5850
Epoch 7/10
1600/1600 [==============================] - 0s 286us/step - loss: 0.5384 - acc: 0.8969 - val_loss: 0.6670 - val_acc: 0.5925
E

In [8]:
model.evaluate(X_train, y_train)

1600/1600 [==============================] - 0s 32us/step


[0.23905182480812073, 0.9975]

In [9]:
model.evaluate(X_test, y_test)

400/400 [==============================] - 0s 38us/step


[0.6549072742462159, 0.64]